In [5]:
#importing necessary libraries
import cv2 as cv
import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def nothing(x):
    pass

cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

# Create a window named trackbars.
cv2.namedWindow("Trackbars")

# Now create 6 tracbars that will control the lower and upper range of H,S & V channels.
# The Arguments are like this: Name of trackbar, window name, range, callback function.
# For Hue the range is 0-179 and for S,V its 0-255.
cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)
 
 
while True:
    
    # Start reading the webcam feed frame by frame.
    ret, frame = cap.read()
    if not ret:
        break
    # Flip the frame horizontally (Not required)
    frame = cv2.flip( frame, 1 ) 
    
    # Convert the BGR image to HSV image.
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Get the new values of the trackbar in real time as the user changes them
    l_h = cv2.getTrackbarPos("L - H", "Trackbars")
    l_s = cv2.getTrackbarPos("L - S", "Trackbars")
    l_v = cv2.getTrackbarPos("L - V", "Trackbars")
    u_h = cv2.getTrackbarPos("U - H", "Trackbars")
    u_s = cv2.getTrackbarPos("U - S", "Trackbars")
    u_v = cv2.getTrackbarPos("U - V", "Trackbars")
 
    # Set the lower and upper HSV range according to the value selected by the trackbar
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])
    
    # Filter the image and get the binary mask, where white represents your target color
    mask = cv2.inRange(hsv, lower_range, upper_range)
 
    # You can also visualize the real part of the target color (Optional)
    res = cv2.bitwise_and(frame, frame, mask=mask)
    
    # Converting the binary mask to 3 channel image, this is just so we can stack it with the others
    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # stack the mask, orginal frame and the filtered result
    stacked = np.hstack((mask_3,frame,res))
    
    # Show this stacked frame at 40% of the size.
    cv2.imshow('Trackbars',cv2.resize(stacked,None,fx=0.4,fy=0.4))
    
    # If the user presses ESC then exit the program
    key = cv2.waitKey(1)
    if key == 27:
        break
    
    # If the user presses `s` then print this array.
    if key == ord('s'):
        
        thearray = [[l_h,l_s,l_v],[u_h, u_s, u_v]]
        print(thearray)

    
# Release the camera & destroy the windows.    
cap.release()
cv2.destroyAllWindows()

In [14]:
# Before we start detecting via webcam , we first 
#need to define a base frame from where the line will start.
global ox,oy,lb,ub
cap=cv.VideoCapture(0)
ret,frame=cap.read()
img=frame.copy()
hsv=cv.cvtColor(frame,cv.COLOR_BGR2HSV)
lb= np.array(thearray)[0]
ub= np.array(thearray)[1]
kernel = np.ones((5, 5), np.uint8)
mask=cv.inRange(hsv,lb,ub)
    #mask = cv2.erode(mask, kernel, iterations=2)
    #mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    # mask=cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)
    #mask = cv2.dilate(mask, kernel, iterations=1)
res = cv2.bitwise_and(img, img, mask=mask)
    
cnts, heir = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
old= np.zeros_like(frame)
    
if len(cnts) >= 1:
    cnt = max(cnts, key=cv2.contourArea)
    ((ox, oy), oradius) = cv2.minEnclosingCircle(cnt)
    cv.circle(img,(int(ox),int(oy)),int(oradius), (0, 255, 255), 2)
    cv.imshow("img",img)

# now using the web cam footage and proecssing it in frames.
while True:
    ret,frame=cap.read()
    img=frame.copy()
    img=cv.flip(img,1)
    frame=cv.flip(frame,1)
    hsv=cv.cvtColor(frame,cv.COLOR_BGR2HSV)
    lb= np.array(thearray)[0]
    ub= np.array(thearray)[1]
    kernel = np.ones((5, 5), np.uint8)
    mask=cv.inRange(hsv,lb,ub)
    #mask = cv2.erode(mask, kernel, iterations=2)
    #mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    # mask=cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)
    #mask = cv2.dilate(mask, kernel, iterations=1)
    res = cv2.bitwise_and(img, img, mask=mask)
    
    cnts, heir = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    
    if len(cnts) >= 1:
        cnt = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        cv.line(old,(int(ox),int(oy)),(int(x),int(y)),(0,255,0),3)
        cv.circle(img,(int(x),int(y)),int(radius), (0, 255, 255), 2)
        final=cv.add(img,old)
        cv.imshow("img",final)
    else:
        final=cv.add(img,old)
        cv.imshow("img",final)
    ox,oy=x,y
            
            
    
    if cv.waitKey(1) ==27:
        break

# When everything is done, release the capture
cap.release()
cv.destroyAllWindows()


In [12]:
cap.release()
cv.destroyAllWindows()